In [5]:
import csv
import pandas as pd
import numpy as np
import pymysql.cursors

database = "healthCC"

def create_database(db_file):
    try:
        conn = pymysql.connect(host='localhost',
                         user='root',
                         password='root')
        conn.cursor().execute('CREATE DATABASE IF NOT EXISTS '+db_file)
        
        return True
    except pymysql.InternalError as e:
        return False
    
def create_connection(db_file):
    """ create a database connection to the MySQL database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',
                             db=db_file)
        return connection
    except pymysql.InternalError as e:
        return False

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except pymysql.InternalError as e:
        print(e)

## Create table queries
create_hospital_table = """ CREATE TABLE IF NOT EXISTS hospitals (
                                        provider_id INTEGER PRIMARY KEY,
                                        name TEXT NOT NULL,
                                        type TEXT NOT NULL,
                                        phone TEXT,
                                        rating INTEGER
                                    ); """
create_address_table = """ CREATE TABLE IF NOT EXISTS address (
                                        id INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
                                        address TEXT NOT NULL,
                                        state TEXT,
                                        city TEXT,
                                        zip TEXT NOT NULL,
                                        county TEXT
                                    ); """
create_ownership_type_table = """ CREATE TABLE IF NOT EXISTS ownership_type (
                                        id INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
                                        ow_type TEXT NOT NULL
                                    ); """
create_hospital_address_table = """ CREATE TABLE IF NOT EXISTS hospital_address (
                                        hospital_id INTEGER NOT NULL,
                                        address_id INTEGER NOT NULL,
                                        PRIMARY KEY (hospital_id, address_id)
                                    ); """
create_hospital_ownership_table = """ CREATE TABLE IF NOT EXISTS hospital_ownership (
                                        hospital_id INTEGER NOT NULL,
                                        ownership_id INTEGER NOT NULL,
                                        PRIMARY KEY (hospital_id, ownership_id)
                                    ); """
create_hospital_comparison_table = """ CREATE TABLE IF NOT EXISTS hospital_comparison (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        hospital_id INTEGER NOT NULL,
                                        emergency_services INTEGER,
                                        ehr INTEGER, 
                                        mortality INTEGER, 
                                        safety INTEGER, 
                                        readmission INTEGER, 
                                        patient_experience INTEGER,
                                        effectiveness INTEGER,
                                        timeliness INTEGER,
                                        medical_imaging INTEGER
                                    ); """


# Insert queries

def insert_hospital(cur, hospital_data):
    """ insert into hospital table 
    :param cur: cursor
    :param hospital_data: hospital data
    :return: current inserted row id
    """
    ## 'Provider ID','Hospital Name','Hospital Type', 'Phone Number', 'Hospital overall rating'
    hp_sql = """INSERT IGNORE INTO hospitals(provider_id, name, type, phone, rating)
            VALUES(%s,%s,%s,%s,%s)"""
    hp_data=(int(hospital_data['Provider ID']),
          hospital_data['Hospital Name'],
          hospital_data['Hospital Type'],
          hospital_data['Phone Number'],
          hospital_data['Hospital overall rating'])
    
    cur.execute(hp_sql, hp_data)
    
    return int(hospital_data['Provider ID'])

def insert_address(cur, address_data):
    """ insert into address table 
    :param cur: cursor
    :param address_data: address data
    :return: current inserted row id
    """
    ## 'Address','State','City', 'Zip Code', 'County Name'
    add_sql = """INSERT INTO address(address, state, city, zip, county) 
                VALUES(%s,%s,%s,%s,%s)"""
    add_data = (address_data['Address'],
               address_data['State'],
               address_data['City'],
               address_data['ZIP Code'],
               address_data['County Name'])

    cur.execute(add_sql, add_data)
    
    return cur.lastrowid

def insert_hospital_address(conn, hospital_id, address_id):
    """ insert into hospital_address table 
    :param cur: cursor
    :param hospital_id: hospital id
    :param address_id: address id
    :return: current inserted row id
    """
    ## 'Hospital Id','Address Id'
    hp_add_sql = """INSERT IGNORE INTO hospital_address(hospital_id, address_id) 
                    VALUES(%s,%s)""" 
    hp_add_data = (int(hospital_id), int(address_id))
    
    cur.execute(hp_add_sql, hp_add_data)
    
    return cur.lastrowid

def insert_ownership_type(cur, ownership_data):
    """ insert into ownership type table 
    :param cur: cursor
    :param ownership_data: ownership data
    :return: current inserted row id
    """
    ## 'Ownership Type'
    select_data = (ownership_data,)
    cur.execute("""SELECT * FROM ownership_type WHERE ow_type=%s""", select_data)
    entry = cur.fetchone()

    if entry is None:
        insert_data = (ownership_data,)
        cur.execute("""INSERT INTO ownership_type(ow_type)
                        VALUES(%s)""", insert_data)
        lasrowid=cur.lastrowid
    else:
        lasrowid=entry[0]
    
    return lasrowid

def insert_hospital_ownership(cur, hospital_id, ow_id):
    """ insert into hospital_ownership table 
    :param cur: cursor
    :param hospital_id: hospital id
    :param ownership_id: ownership_id
    :return: current inserted row id
    """
    ## insert into hospital_ownership
    hp_ow_sql = """INSERT IGNORE INTO hospital_ownership(hospital_id, ownership_id) 
                    VALUES(%s,%s)""" 
    hp_ow_data = (int(hospital_id), int(ow_id))
    
    cur.execute(hp_ow_sql, hp_ow_data)
    
    return cur.lastrowid

def insert_hospital_comparison(cur, hp_comp_data, hp_id):
    """ insert into hospital_comparison table 
    :param cur: cursor
    :param hp_comp_data: hospital comparison data
    :param hp_id: hospital id 
    :return: current inserted row id
    """
    ## insert into hospital_comparison
    hp_comp_sql = """INSERT INTO hospital_comparison(hospital_id, emergency_services, ehr, mortality, safety, readmission, 
                        patient_experience, effectiveness, timeliness, medical_imaging) 
                    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    hp_comp_data = (hp_id, 
                    hp_comp_data['Emergency Services'],
                    hp_comp_data['Meets criteria for meaningful use of EHRs'],
                    hp_comp_data['Mortality national comparison'],
                    hp_comp_data['Safety of care national comparison'],
                    hp_comp_data['Readmission national comparison'],
                    hp_comp_data['Patient experience national comparison'],
                    hp_comp_data['Effectiveness of care national comparison'],
                    hp_comp_data['Timeliness of care national comparison'],
                    hp_comp_data['Efficient use of medical imaging national comparison'])
    
    cur.execute(hp_comp_sql, hp_comp_data)
    
    return cur.lastrowid

# create database
res = create_database(database)
# create a database connection
if res==True:
    conn = create_connection(database)

    if conn is not None:
        # create all tables
        create_table(conn, create_hospital_table)
        create_table(conn, create_address_table)
        create_table(conn, create_ownership_type_table)
        create_table(conn, create_hospital_address_table)
        create_table(conn, create_hospital_ownership_table)
        create_table(conn, create_hospital_comparison_table)
    else:
        print("Error! cannot create the database connection.")

    # Commit connection and close cursor and connection
    conn.commit()
    conn.close()

    conn = create_connection(database)


    chunksize = 500
    cur = conn.cursor()
    for df in pd.read_csv('data/hospital_gen_info.csv', chunksize=chunksize, iterator=True):
        #County ## Replace 'nan' = ''
        County_mapping={np.NaN:''}
        df['County Name'].replace(County_mapping, inplace=True)
        # Hospital Type ## Replace 'Acute Care Hospitals'=1, 'Critical Access Hospitals'=2, 'Childrens'=3
        HType_mapping={'Acute Care Hospitals':1, 'Critical Access Hospitals':2, 'Childrens':3}
        df['Hospital Type'].replace(HType_mapping, inplace=True)
        # 'Meets criteria for meaningful use of EHRs' ## Replace Y=1, nan=-100
        EHR_mapping={'Y':1, np.NaN:-100}
        df['Meets criteria for meaningful use of EHRs'].replace(EHR_mapping, inplace=True)

        # 'Emergency Services' ## Replace Yes=1, No=0, 'nan'=-100
        ES_mapping = {'Yes': 1, 'No': 0, np.NaN:-100}
        df['Emergency Services'].replace(ES_mapping, inplace=True)

        # 'Hospital overall rating ## Replace Not Available as -100 
        df['Hospital overall rating'].replace({'Not Available':-100}, inplace=True)

        # Same as the national average = 0, Below the national average = -1, Above the national average = 1, Not Available = -100
        MC_mapping = {'Same as the national average' : 0,
                      'Below the national average' : -1, 
                      'Above the national average' : 1, 
                      'Not Available' : -100 }

        df['Mortality national comparison'].replace(MC_mapping, inplace=True)
        df['Safety of care national comparison'].replace(MC_mapping, inplace=True)
        df['Readmission national comparison'].replace(MC_mapping, inplace=True)
        df['Patient experience national comparison'].replace(MC_mapping, inplace=True)
        df['Effectiveness of care national comparison'].replace(MC_mapping, inplace=True)
        df['Timeliness of care national comparison'].replace(MC_mapping, inplace=True)
        df['Efficient use of medical imaging national comparison'].replace(MC_mapping, inplace=True)

        for index, row in df.iterrows():
            hospital_id = insert_hospital(cur, row)
            print("hid: "+str(hospital_id))
            address_id = insert_address(cur, row)
            print("aid: "+str(address_id))

            insert_hospital_address(cur, hospital_id, address_id)

            ow_id = insert_ownership_type(cur, row['Hospital Ownership'])
            
            insert_hospital_ownership(cur, hospital_id, ow_id)
            #print(row)
            insert_hospital_comparison(cur, row, hospital_id)
            
    conn.commit()
    conn.close()
       

hid: 10001
aid: 1
hid: 10005
aid: 2
hid: 10006
aid: 3
hid: 10007
aid: 4
hid: 10008
aid: 5
hid: 10011
aid: 6
hid: 10012
aid: 7
hid: 10016
aid: 8
hid: 10018
aid: 9
hid: 10019
aid: 10
hid: 10021
aid: 11
hid: 10022
aid: 12
hid: 10023
aid: 13
hid: 10024
aid: 14
hid: 10029
aid: 15
hid: 10032
aid: 16
hid: 10033
aid: 17
hid: 10034
aid: 18
hid: 10035
aid: 19
hid: 10036
aid: 20
hid: 10038
aid: 21
hid: 10039
aid: 22
hid: 10040
aid: 23
hid: 10044
aid: 24
hid: 10045
aid: 25
hid: 10046
aid: 26
hid: 10047
aid: 27
hid: 10049
aid: 28
hid: 10051
aid: 29
hid: 10052
aid: 30
hid: 10055
aid: 31
hid: 10056
aid: 32
hid: 10058
aid: 33
hid: 10059
aid: 34
hid: 10061
aid: 35
hid: 10062
aid: 36
hid: 10065
aid: 37
hid: 10069
aid: 38
hid: 10073
aid: 39
hid: 10078
aid: 40
hid: 10079
aid: 41
hid: 10083
aid: 42
hid: 10085
aid: 43
hid: 10086
aid: 44
hid: 10087
aid: 45
hid: 10089
aid: 46
hid: 10090
aid: 47
hid: 10091
aid: 48
hid: 10092
aid: 49
hid: 10095
aid: 50
hid: 10097
aid: 51
hid: 10099
aid: 52
hid: 10100
aid: 53
hi

hid: 50378
aid: 422
hid: 50380
aid: 423
hid: 50382
aid: 424
hid: 50390
aid: 425
hid: 50393
aid: 426
hid: 50394
aid: 427
hid: 50396
aid: 428
hid: 50397
aid: 429
hid: 50407
aid: 430
hid: 50411
aid: 431
hid: 50414
aid: 432
hid: 50417
aid: 433
hid: 50423
aid: 434
hid: 50424
aid: 435
hid: 50425
aid: 436
hid: 50426
aid: 437
hid: 50438
aid: 438
hid: 50441
aid: 439
hid: 50444
aid: 440
hid: 50454
aid: 441
hid: 50455
aid: 442
hid: 50457
aid: 443
hid: 50464
aid: 444
hid: 50468
aid: 445
hid: 50471
aid: 446
hid: 50481
aid: 447
hid: 50485
aid: 448
hid: 50488
aid: 449
hid: 50492
aid: 450
hid: 50496
aid: 451
hid: 50498
aid: 452
hid: 50502
aid: 453
hid: 50503
aid: 454
hid: 50506
aid: 455
hid: 50510
aid: 456
hid: 50512
aid: 457
hid: 50515
aid: 458
hid: 50516
aid: 459
hid: 50517
aid: 460
hid: 50523
aid: 461
hid: 50526
aid: 462
hid: 50528
aid: 463
hid: 50534
aid: 464
hid: 50537
aid: 465
hid: 50541
aid: 466
hid: 50543
aid: 467
hid: 50546
aid: 468
hid: 50547
aid: 469
hid: 50548
aid: 470
hid: 50549
aid: 471


hid: 100256
aid: 859
hid: 100258
aid: 860
hid: 100259
aid: 861
hid: 100260
aid: 862
hid: 100264
aid: 863
hid: 100265
aid: 864
hid: 100266
aid: 865
hid: 100267
aid: 866
hid: 100268
aid: 867
hid: 100269
aid: 868
hid: 100275
aid: 869
hid: 100276
aid: 870
hid: 100277
aid: 871
hid: 100281
aid: 872
hid: 100284
aid: 873
hid: 100285
aid: 874
hid: 100286
aid: 875
hid: 100287
aid: 876
hid: 100288
aid: 877
hid: 100289
aid: 878
hid: 100290
aid: 879
hid: 100291
aid: 880
hid: 100292
aid: 881
hid: 100296
aid: 882
hid: 100298
aid: 883
hid: 100299
aid: 884
hid: 100302
aid: 885
hid: 100307
aid: 886
hid: 100313
aid: 887
hid: 100314
aid: 888
hid: 100315
aid: 889
hid: 100316
aid: 890
hid: 100319
aid: 891
hid: 100320
aid: 892
hid: 100321
aid: 893
hid: 100322
aid: 894
hid: 100327
aid: 895
hid: 100328
aid: 896
hid: 100329
aid: 897
hid: 101300
aid: 898
hid: 101301
aid: 899
hid: 101302
aid: 900
hid: 101303
aid: 901
hid: 101304
aid: 902
hid: 101305
aid: 903
hid: 101307
aid: 904
hid: 101308
aid: 905
hid: 101309
a

hid: 140307
aid: 1245
hid: 141300
aid: 1246
hid: 141301
aid: 1247
hid: 141302
aid: 1248
hid: 141303
aid: 1249
hid: 141304
aid: 1250
hid: 141305
aid: 1251
hid: 141306
aid: 1252
hid: 141307
aid: 1253
hid: 141308
aid: 1254
hid: 141309
aid: 1255
hid: 141310
aid: 1256
hid: 141311
aid: 1257
hid: 141312
aid: 1258
hid: 141313
aid: 1259
hid: 141315
aid: 1260
hid: 141316
aid: 1261
hid: 141317
aid: 1262
hid: 141318
aid: 1263
hid: 141319
aid: 1264
hid: 141320
aid: 1265
hid: 141321
aid: 1266
hid: 141322
aid: 1267
hid: 141323
aid: 1268
hid: 141324
aid: 1269
hid: 141325
aid: 1270
hid: 141326
aid: 1271
hid: 141327
aid: 1272
hid: 141328
aid: 1273
hid: 141329
aid: 1274
hid: 141330
aid: 1275
hid: 141331
aid: 1276
hid: 141332
aid: 1277
hid: 141333
aid: 1278
hid: 141334
aid: 1279
hid: 141335
aid: 1280
hid: 141336
aid: 1281
hid: 141337
aid: 1282
hid: 141338
aid: 1283
hid: 141339
aid: 1284
hid: 141340
aid: 1285
hid: 141341
aid: 1286
hid: 141342
aid: 1287
hid: 141343
aid: 1288
hid: 141344
aid: 1289
hid: 14134

hid: 171352
aid: 1639
hid: 171353
aid: 1640
hid: 171354
aid: 1641
hid: 171355
aid: 1642
hid: 171356
aid: 1643
hid: 171357
aid: 1644
hid: 171358
aid: 1645
hid: 171359
aid: 1646
hid: 171360
aid: 1647
hid: 171361
aid: 1648
hid: 171362
aid: 1649
hid: 171363
aid: 1650
hid: 171364
aid: 1651
hid: 171365
aid: 1652
hid: 171366
aid: 1653
hid: 171367
aid: 1654
hid: 171368
aid: 1655
hid: 171369
aid: 1656
hid: 171370
aid: 1657
hid: 171371
aid: 1658
hid: 171372
aid: 1659
hid: 171373
aid: 1660
hid: 171374
aid: 1661
hid: 171375
aid: 1662
hid: 171376
aid: 1663
hid: 171377
aid: 1664
hid: 171378
aid: 1665
hid: 171379
aid: 1666
hid: 171380
aid: 1667
hid: 171381
aid: 1668
hid: 171382
aid: 1669
hid: 171383
aid: 1670
hid: 171384
aid: 1671
hid: 173300
aid: 1672
hid: 180001
aid: 1673
hid: 180002
aid: 1674
hid: 180004
aid: 1675
hid: 180005
aid: 1676
hid: 180009
aid: 1677
hid: 180010
aid: 1678
hid: 180011
aid: 1679
hid: 180012
aid: 1680
hid: 180013
aid: 1681
hid: 180016
aid: 1682
hid: 180017
aid: 1683
hid: 18001

...

aid: 3219
hid: 360151
aid: 3220
hid: 360152
aid: 3221
hid: 360153
aid: 3222
hid: 360155
aid: 3223
hid: 360156
aid: 3224
hid: 360159
aid: 3225
hid: 360161
aid: 3226
hid: 360163
aid: 3227
hid: 360170
aid: 3228
hid: 360172
aid: 3229
hid: 360174
aid: 3230
hid: 360175
aid: 3231
hid: 360179
aid: 3232
hid: 360180
aid: 3233
hid: 360185
aid: 3234
hid: 360189
aid: 3235
hid: 360192
aid: 3236
hid: 360197
aid: 3237
hid: 360203
aid: 3238
hid: 360210
aid: 3239
hid: 360211
aid: 3240
hid: 360218
aid: 3241
hid: 360230
aid: 3242
hid: 360234
aid: 3243
hid: 360236
aid: 3244
hid: 360239
aid: 3245
hid: 360241
aid: 3246
hid: 360245
aid: 3247
hid: 360247
aid: 3248
hid: 360259
aid: 3249
hid: 360261
aid: 3250
hid: 360262
aid: 3251
hid: 360263
aid: 3252
hid: 360266
aid: 3253
hid: 360270
aid: 3254
hid: 360274
aid: 3255
hid: 360276
aid: 3256
hid: 360348
aid: 3257
hid: 360351
aid: 3258
hid: 360352
aid: 3259
hid: 360354
aid: 3260
hid: 360355
aid: 3261
hid: 360358
aid: 3262
hid: 360359
aid: 3263
hid: 360360
aid: 3264


hid: 390225
aid: 3608
hid: 390226
aid: 3609
hid: 390228
aid: 3610
hid: 390231
aid: 3611
hid: 390233
aid: 3612
hid: 390236
aid: 3613
hid: 390237
aid: 3614
hid: 390256
aid: 3615
hid: 390258
aid: 3616
hid: 390263
aid: 3617
hid: 390265
aid: 3618
hid: 390266
aid: 3619
hid: 390267
aid: 3620
hid: 390268
aid: 3621
hid: 390270
aid: 3622
hid: 390272
aid: 3623
hid: 390278
aid: 3624
hid: 390290
aid: 3625
hid: 390302
aid: 3626
hid: 390304
aid: 3627
hid: 390307
aid: 3628
hid: 390312
aid: 3629
hid: 390314
aid: 3630
hid: 390316
aid: 3631
hid: 390321
aid: 3632
hid: 390322
aid: 3633
hid: 390323
aid: 3634
hid: 390324
aid: 3635
hid: 390325
aid: 3636
hid: 390326
aid: 3637
hid: 390327
aid: 3638
hid: 390328
aid: 3639
hid: 390329
aid: 3640
hid: 390330
aid: 3641
hid: 391300
aid: 3642
hid: 391301
aid: 3643
hid: 391302
aid: 3644
hid: 391303
aid: 3645
hid: 391304
aid: 3646
hid: 391305
aid: 3647
hid: 391306
aid: 3648
hid: 391307
aid: 3649
hid: 391308
aid: 3650
hid: 391309
aid: 3651
hid: 391312
aid: 3652
hid: 39131

hid: 450132
aid: 4001
hid: 450133
aid: 4002
hid: 450135
aid: 4003
hid: 450137
aid: 4004
hid: 450143
aid: 4005
hid: 450144
aid: 4006
hid: 450147
aid: 4007
hid: 450148
aid: 4008
hid: 450152
aid: 4009
hid: 450154
aid: 4010
hid: 450155
aid: 4011
hid: 450162
aid: 4012
hid: 450163
aid: 4013
hid: 450165
aid: 4014
hid: 450176
aid: 4015
hid: 450184
aid: 4016
hid: 450187
aid: 4017
hid: 450192
aid: 4018
hid: 450193
aid: 4019
hid: 450194
aid: 4020
hid: 450196
aid: 4021
hid: 450200
aid: 4022
hid: 450203
aid: 4023
hid: 450209
aid: 4024
hid: 450210
aid: 4025
hid: 450211
aid: 4026
hid: 450213
aid: 4027
hid: 450219
aid: 4028
hid: 450222
aid: 4029
hid: 450229
aid: 4030
hid: 450231
aid: 4031
hid: 450235
aid: 4032
hid: 450236
aid: 4033
hid: 450237
aid: 4034
hid: 450241
aid: 4035
hid: 450243
aid: 4036
hid: 450253
aid: 4037
hid: 450271
aid: 4038
hid: 450272
aid: 4039
hid: 450280
aid: 4040
hid: 450289
aid: 4041
hid: 450292
aid: 4042
hid: 450293
aid: 4043
hid: 450299
aid: 4044
hid: 450306
aid: 4045
hid: 45032

hid: 490044
aid: 4378
hid: 490045
aid: 4379
hid: 490046
aid: 4380
hid: 490048
aid: 4381
hid: 490050
aid: 4382
hid: 490052
aid: 4383
hid: 490053
aid: 4384
hid: 490057
aid: 4385
hid: 490059
aid: 4386
hid: 490060
aid: 4387
hid: 490063
aid: 4388
hid: 490066
aid: 4389
hid: 490067
aid: 4390
hid: 490069
aid: 4391
hid: 490075
aid: 4392
hid: 490077
aid: 4393
hid: 490079
aid: 4394
hid: 490084
aid: 4395
hid: 490088
aid: 4396
hid: 490089
aid: 4397
hid: 490090
aid: 4398
hid: 490092
aid: 4399
hid: 490093
aid: 4400
hid: 490094
aid: 4401
hid: 490097
aid: 4402
hid: 490098
aid: 4403
hid: 490101
aid: 4404
hid: 490104
aid: 4405
hid: 490107
aid: 4406
hid: 490110
aid: 4407
hid: 490111
aid: 4408
hid: 490112
aid: 4409
hid: 490113
aid: 4410
hid: 490114
aid: 4411
hid: 490115
aid: 4412
hid: 490116
aid: 4413
hid: 490117
aid: 4414
hid: 490118
aid: 4415
hid: 490119
aid: 4416
hid: 490120
aid: 4417
hid: 490122
aid: 4418
hid: 490126
aid: 4419
hid: 490127
aid: 4420
hid: 490129
aid: 4421
hid: 490130
aid: 4422
hid: 49013

hid: 670085
aid: 4775
hid: 670087
aid: 4776
hid: 670088
aid: 4777
hid: 670090
aid: 4778
hid: 670092
aid: 4779
hid: 670093
aid: 4780
hid: 670094
aid: 4781
hid: 670095
aid: 4782
hid: 670096
aid: 4783
hid: 670097
aid: 4784
hid: 670098
aid: 4785
hid: 670099
aid: 4786
hid: 670102
aid: 4787
hid: 670103
aid: 4788
hid: 670106
aid: 4789
hid: 670107
aid: 4790
hid: 670108
aid: 4791
hid: 670109
aid: 4792
hid: 670110
aid: 4793
hid: 670112
aid: 4794
hid: 670113
aid: 4795
hid: 670114
aid: 4796
hid: 670115
aid: 4797
hid: 670116
aid: 4798
hid: 670117
aid: 4799
hid: 670118
aid: 4800
hid: 670119
aid: 4801
hid: 670120
aid: 4802
hid: 640001
aid: 4803
hid: 650001
aid: 4804
hid: 650003
aid: 4805
hid: 660001
aid: 4806
